# Numerical Methods for Stochastic Differential Equations

This is a lab on Matthew's Chapter 7.

In [ ]:
# import the relevant libraries.

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import random

## Random Processes

Review of some random processes, e.g. random walk, Wiener process, etc.

### Random Walk

A random walk is a discrete stochastic process where the state $X_{n+1}$ is related to the state $X_n$ in that they differ by a distance $1$.

In [ ]:
# random walk in one dimension.
def random_walk_one_dimension(n, x_init):
    
    # type testing.
    if not isinstance(n, int):
        raise TypeError('n must be int.')
        pass
    
    def random_walk_iter(x):
        return x + random.choice([-1, 1])
    
    solution = [None for _ in range(n)]
    for i in range(n):
        if i == 0:
            solution[0] = x_init
            pass
        else:
            solution[i] = random_walk_iter(solution[i-1])
            pass
        pass
    return solution

In [ ]:
n = 100
plt.plot(range(n), random_walk_one_dimension(n, 0))
plt.title('Random Walk')
plt.show()

### Wiener Process

A Wiener Process is a random process $Y(t_{k+1}) = Y(t_k) + \sqrt{\delta t}J_k$.

In [ ]:
# Wiener process in one dimension.
def wiener_process_one_dimension(tt, y_init):
    # tt is a linear array.
    
    # extract data from linear array.
    n = len(tt)
    t_min = tt[0]
    t_max = tt[n-1]
    dt = (t_max - t_min) / (n - 1)
    
    # construct solution set.
    solution = [None for _ in range(n)]
    
    # solve.
    for i in range(n):
        if i == 0:
            solution[0] = y_init
            pass
        else:
            solution[i] = solution[i-1] + np.sqrt(dt)*random.choice([-1, 1])
            pass
        pass
    return solution

In [ ]:
tt = np.linspace(0, 1, 1000)
plt.plot(tt, wiener_process_one_dimension(tt, 0))
plt.title('Wiener Process with 1000 Timesteps')
plt.show()

### Uniformly random on the surface of a unit $n$-ball

Let $\mathbf{x}$ be an $n$-dimensional vector of normal deviates and $r = \vert\vert x\vert\vert$. Then the vector $\mathbf{x}/r$ is randomly chosen over the surface of an $n$-sphere. (Marsaglia)

In [ ]:
def norm(x):
    return np.sqrt(x.dot(x))

def unit_sphere(n):
    x = np.array([random.normalvariate(0, 1) for _ in range(n)])
    r = norm(x)
    
    return x/r

In [ ]:
# randomly selected points on a ball.

n = 1000
points = [unit_sphere(3) for _ in range(n)]

xx = [point[0] for point in points]
yy = [point[1] for point in points]
zz = [point[2] for point in points]

In [ ]:
# plot them in three dimensions.

plt.figure(figsize = (10, 10))
ax = plt.axes(projection = '3d')
ax.scatter(xx, yy, zz)
plt.title('Random Points on Surface of Ball')
plt.show()

### Generalized Wiener Process

In [ ]:
# wiener process in n dimensions.

def wiener_process(tt, x_init):
    
    # calculate dimension.
    dim = len(x_init)
    
    # extract data from tt.
    n = len(tt)
    t_min = tt[0]
    t_max = tt[n-1]
    dt = (t_max - t_min) / (n - 1)
    
    # construct solution set.
    solution = [None for _ in range(n)]
    
    # solve.
    for i in range(n):
        if i == 0:
            solution[0] = x_init
            pass
        else:
            solution[i] = solution[i-1] + np.sqrt(dt)*unit_sphere(dim)
            pass
        pass
    return solution

In [ ]:
tt = np.linspace(0, 1, 1000)

# a two-dimensional Wiener process.
solution = wiener_process(tt, np.array([0, 0]))
xx = [s[0] for s in solution]
yy = [s[1] for s in solution]

plt.figure(figsize = (10, 10))
plt.plot(xx, yy)
plt.title('2D Wiener Process')
plt.show()

# a three-dimensional Wiener process
solution = wiener_process(tt, np.array([0, 0, 0]))
xx = [s[0] for s in solution]
yy = [s[1] for s in solution]
zz = [s[2] for s in solution]

plt.figure(figsize = (10, 10))
ax = plt.axes(projection = '3d')
ax.plot3D(xx, yy, zz)
plt.title('3D Wiener Process')
plt.show()

Let $m_i = i$ and $p_i = i^2$ for $i = 1, 2, 3$.

In [ ]:
mass_matrix = np.array([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 3]
])

momentum = np.array([[np.power(i, 2)] for i in range(1, 4)])

Compose $\mathbf{M}\cdot\mathbf{p}$.

In [ ]:
mass_matrix.dot(momentum)

Compose $\mathbf{M}^{-1}\cdot\mathbf{p}$

In [ ]:
np.linalg.inv(mass_matrix).dot(momentum)

## Methods

In [ ]:
def solve(constructor, initial_condition, tt):
    # extract data from tt.
    n = len(tt)
    t_min, t_max = tt[0], tt[n-1]
    h = (t_max - t_min) / (n - 1)
    
    # method function.
    method_function = constructor(h)
    
    # create vector solution set.
    solution = [None for _ in range(n)]
    
    # compute solutions.
    for i, t in enumerate(tt):
        if i == 0:
            solution[0] = initial_condition
            pass
        else:
            solution[i] = method_function(t, solution[i-1])
            pass
        pass
    return solution

### Euler Maruyama

The Euler Maruyama method for a stochastic differential equation $dX = a(X)dt + b(X)dW$ is $X_{n+1} = X_n + h\cdot a(X_n) + b(X_n)\sqrt{h}R_n$, where $R_n$ is a normal random variable with zero mean and $1$ variance.

In [ ]:
def euler_maruyama_constructor(ff):
    a, b = ff
    def euler_maruyama_stepsize(h):
        def euler_maruyama_method(t, state):
            return state + h*a(state) + b(state)*np.sqrt(h)*random.normalvariate(0, 1)
        
        return euler_maruyama_method
    return euler_maruyama_stepsize

Examine with the SDE $dX = -Xdt + XdW$.

In [ ]:
def a(x):
    return -x

def b(x):
    return x

In [ ]:
# set up problem.
ff = [a, b]
constructor = euler_maruyama_constructor(ff)
initial_condition = 10
tt = np.linspace(0, 1, 10000)

In [ ]:
solution = solve(constructor, initial_condition, tt)

In [ ]:
plt.plot(tt, solution)
plt.title('SDE: $dX = -Xdt + XdW$')
plt.show()

## Sampling Iterations